d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

#Create Delta Tables
 
Objective: Convert a Parquet-based table to a Delta table. 

Recall that a Delta table consists of three things:
- the data files kept in object storage (i.e. AWS S3, Azure Data Lake Storage)
- the Delta Transaction Log saved with the data files in object storage
- a table registered in the Metastore. This step is optional, but usually recommended.

## Notebook Configuration

Before you run this cell, make sure to add a unique user name to the file
<a href="$./includes/configuration" target="_blank">
includes/configuration</a>, e.g.

```
username = "yourfirstname_yourlastname"
```

In [0]:
%run ./includes/configuration

Out[3]: DataFrame[]

## Creating a Table
With Delta Lake, you create tables:
* When ingesting new files into a Delta Table for the first time
* By transforming an existing Parquet-based data lake table to a Delta table

**NOTE:**  Throughout this section, we'll be writing files to the root location of the Databricks File System (DBFS).
In general, best practice is to write files to your cloud object storage.  We use DBFS root here for demonstration purposes.

-sandbox

#### Step 1: Describe the `health_tracker_processed` Table
Before we convert the `health_tracker_processed` table, let's use the Spark SQL `DESCRIBE`command, with the optional parameter `EXTENDED`, to display the attributes of the table.
Note that the table has the "provider" listed as `PARQUET`.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> You will have to scroll to the `#Detailed Table Information` to find the provider.

In [0]:
%sql

DESCRIBE EXTENDED health_tracker_processed

col_name,data_type,comment
dte,date,null
time,timestamp,null
heartrate,double,null
name,string,null
p_device_id,int,null
# Partition Information,,
# col_name,data_type,comment
p_device_id,int,null
,,
# Detailed Table Information,,


### Convert an Existing Parquet Table to a Delta table

Os arquivos Parquet podem ser convertidos no local em arquivos Delta.

A seguir, converteremos a tabela de data lake baseada em Parquet que criamos anteriormente em uma tabela Delta. Ao fazer isso, estamos definindo a Fonte Única da Verdade no coração de nosso EDSS.
O `health_tracker_processed` será a única fonte da verdade.

<img
     alt="OLAP"
     src=https://files.training.databricks.com/images/delta-lake-hands-on/03_delta_tables-01_olap.jpeg
     width=600px
>

#### Step 1: Convert the Files to Delta Files

First, we'll convert the files in-place to Delta files. The conversion creates a Delta Lake transaction log that tracks associated files.

In [0]:
from delta.tables import DeltaTable

parquet_table = f"parquet.`{health_tracker}processed`"
partitioning_scheme = "p_device_id int"

DeltaTable.convertToDelta(spark, parquet_table, partitioning_scheme)

Out[5]: JavaObject id=o376

#### Step 2: Register the Delta Table
At this point, the files containing our records have been converted to Delta files.
The Metastore, however, has not been updated to reflect the change.
To change this we re-register the table in the Metastore.
The Spark SQL command will automatically infer the data schema by reading the footers of the Delta files.

In [0]:
spark.sql(f"""
DROP TABLE IF EXISTS health_tracker_processed
""")

spark.sql(f"""
CREATE TABLE health_tracker_processed
USING DELTA
LOCATION "{health_tracker}/processed" 
""")

Out[6]: DataFrame[]

#### Step 3: Add column comments

Comments can make your tables easier to read and maintain. We use an `ALTER TABLE` command to add new column comments to the exiting Delta table.

In [0]:
%sql
ALTER TABLE
  health_tracker_processed
REPLACE COLUMNS
  (dte DATE COMMENT "Format: YYYY/mm/dd", 
  time TIMESTAMP, 
  heartrate DOUBLE,
  name STRING COMMENT "Format: First Last",
  p_device_id INT COMMENT "range 0 - 4")
  

#### Step 4: Describe the `health_tracker_processed` Table
We can verify that comments have been added to the table by using the `DESCRIBE`Spark SQL command followed by the optional parameter, `EXTENDED`. You can see the column comments that we added as well as some additional information. Scrool down to confirm that the new table had Delta listed as the provider.

In [0]:
%sql
DESCRIBE EXTENDED health_tracker_processed

col_name,data_type,comment
dte,date,Format: YYYY/mm/dd
time,timestamp,
heartrate,double,
name,string,Format: First Last
p_device_id,int,range 0 - 4
,,
# Partitioning,,
Part 0,p_device_id,
,,
# Detailed Table Information,,


#### Step 4: Count the Records in the `health_tracker_processed` table
We count the records in `health_tracker_processed` with Apache Spark.
With Delta Lake, the Delta table requires no repair and is immediately ready for use.

In [0]:
health_tracker_processed = spark.read.table("health_tracker_processed")
health_tracker_processed.count()

Out[7]: 3720

### Create a New Delta Table
Next, we'll create a new Delta table. We'll do this by creating an aggregate table
from the data in the health_track_processed Delta table we just created.
Within the context of our EDSS, this is a downstream aggregate table or data mart.

#### Step 1: Remove files in the `health_tracker_user_analytics` directory
This step will make the notebook idempotent. In other words, it could be run more than once without throwing errors or introducing extra files.

In [0]:
dbutils.fs.rm(health_tracker + "gold/health_tracker_user_analytics",
              recurse=True)

Out[8]: False

#### Step 2: Create an Aggregate DataFrame
The subquery used to define the table is an aggregate query over the `health_tracker_processed` Delta table using summary statistics for each device.

In [0]:
from pyspark.sql.functions import col, avg, max, stddev

health_tracker_gold_user_analytics = (
  health_tracker_processed
  .groupby("p_device_id")
  .agg(avg(col("heartrate")).alias("avg_heartrate"),
       max(col("heartrate")).alias("max_heartrate"),
       stddev(col("heartrate")).alias("stddev_heartrate"))
)

#### Step 3: Write the Delta Files

In [0]:
(health_tracker_gold_user_analytics.write
 .format("delta")
 .mode("overwrite")
 .save(health_tracker + "gold/health_tracker_user_analytics"))

#### Step 4: Register the Delta table in the Metastore
Finally, register this table in the Metastore.

In [0]:
spark.sql(f"""
DROP TABLE IF EXISTS health_tracker_gold_user_analytics
""")

spark.sql(f"""
CREATE TABLE health_tracker_gold_user_analytics
USING DELTA
LOCATION "{health_tracker}/gold/health_tracker_user_analytics"
""")


Out[11]: DataFrame[]

In [0]:
display(health_tracker_gold_user_analytics)

p_device_id,avg_heartrate,max_heartrate,stddev_heartrate
1,78.5776567337699,168.114687819,31.619679037848556
0,81.21484441523789,186.4790827731,31.343789198032887
3,82.65419819635204,171.8435388833,30.929328740004436
4,83.08377376550952,173.5770785921,34.16032267669617
2,79.99574196662837,184.7433209566,31.408007741221997


### Configuring the Visualization
Create a Databricks visualization to view the aggregate sensor data.
We have used the following options to configure the visualization:
```
Keys: p_device_id
Series groupings: None
Values: max_heartrate, avg_heartrate, stddev_heartrate
Aggregation: SUM
Display Type: Bar Chart
```

In [0]:
display(health_tracker_gold_user_analytics)

p_device_id,avg_heartrate,max_heartrate,stddev_heartrate
1,78.5776567337699,168.114687819,31.619679037848556
0,81.21484441523789,186.4790827731,31.343789198032887
3,82.65419819635204,171.8435388833,30.929328740004436
4,83.08377376550952,173.5770785921,34.16032267669617
2,79.99574196662837,184.7433209566,31.408007741221997


-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>